# Outflow Dynamics
Primary notebook for comparing simulations with each other. 
## Utilities

In [ ]:
# Loading libraries and key coordinates
import glob
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.legend_handler import HandlerTuple
from scipy import interpolate
from matplotlib.lines import Line2D
from scipy import stats
import seaborn as sns
from scipy import optimize
from matplotlib.lines import Line2D

# Utilities file
from utilities import sol_in
from utilities import sol_out
from utilities import mean_molecular_weight
from utilities import Temp_S

################################################
##### PHYSICAL CONSTANTS #####
gamma = 5/3
kb = 1.3807e-16 # Boltzmann Constant in CGS or erg/K
seconds_in_myrs = 3.15576e+13
seconds_in_yrs = 3.154e+7
sb_constant_cgs = 5.670374419e-5 # ergs/cm^2/K^4/s
T_COLD_MAX = 3e4
from utilities import HYDROGEN_MASS_FRACTION # 0.76
from utilities import PROTON_MASS_GRAMS # 1.67262192e-24 
from utilities import GRAVITIONAL_CONSTANT_IN_CGS #  6.6738e-8
from utilities import HUBBLE #  3.2407789e-18
from utilities import M_PI #  3.14159265358979323846

# # 2+43 erg*1e5 cm/second/1e21 cm
plt.style.use("seaborn-paper")
##### UNITS #####
UnitVelocity_in_cm_per_s = 1e5 # 10 km/sec  
UnitLength_in_cm = 3.085678e21 # 1 kpc
UnitMass_in_g  = 1.989e33 # 1 solar mass
UnitTime_in_s = UnitLength_in_cm / UnitVelocity_in_cm_per_s # 3.08568e+16 seconds 
UnitEnergy_in_cgs = UnitMass_in_g * pow(UnitLength_in_cm, 2) / pow(UnitTime_in_s, 2) # 1.9889999999999999e+43 erg
UnitDensity_in_cgs = UnitMass_in_g / pow(UnitLength_in_cm, 3) # 6.76989801444063e-32 g/cm^3
UnitPressure_in_cgs = UnitMass_in_g / UnitLength_in_cm / pow(UnitTime_in_s, 2) # 6.769911178294542e-22 barye
UnitNumberDensity = UnitDensity_in_cgs/PROTON_MASS_GRAMS
UnitEnergyDensity = UnitEnergy_in_cgs/pow(UnitLength_in_cm, 3)

G_code = GRAVITIONAL_CONSTANT_IN_CGS/(pow(UnitLength_in_cm,3) * pow(UnitMass_in_g, -1) * pow(UnitTime_in_s, -2)) 
Hubble_code = HUBBLE * UnitTime_in_s # All.Hubble in Arepo

###### KEY SIMULATION PARAMETERS ######
boxsize = 100
midpoint = boxsize/2 
center_boxsize = 10
cells_per_dim = 301 

angle_l = 60
dx = center_boxsize/300
eps = dx/1e6
Z_solar = 0.02

def custom_tick_labels(x, pos):
    return f"{x - boxsize/2:.0f}"

##### DEFINE OUTPUT FILES HERE ######
# Non-Cooling
## Injection Radius
# outputs = ["./cc85_disk/output_center_ref/", "./cc85_disk/output_r300oc"]

## Disk Models
# outputs = ["./cc85_disk/output_center_ref/", "./cc85_disk/output_center_ref_large/", "./cc85_disk/output_center_ref_small/"]

## Injection Radius
# outputs = ["./cc85_disk/output_center_ref/", "./cc85_disk/output_r300pc/"]

## Fast vs Slow Outflow
# outputs = ["./cc85_disk/output_center_ref/", "./cc85_disk/output_center_ref_large/", "./cc85_disk/output_center_ref_small/",]

## Disk Models 
# outputs = ["./cc85_disk/output_center_ref/", "./cc85_disk/output_center_ref_large/", "./cc85_disk/output_center_ref_small/",]

###############################################################

# Cooling
## Metallicities + No-Cooling
# outputs = ["cc85_disk/output_cooling_params/", "./outflow_cooling/output_PIE_fid/", "./outflow_cooling/output_PIE_mZ/", "./outflow_cooling/output_PIE_lZ/"]

# Loading Parameters
outputs = ["./outflow_cooling/output_PIE_fid/", "./outflow_cooling/output_high_alpha/", "./outflow_cooling/output_high_beta/", "./outflow_cooling/output_high_alpha_high_beta/"]
labeling = []

## CIE vs PIE 
# outputs = ["./outflow_cooling/output_PIE_fid/", "./outflow_cooling/output_CIE_fid/"] 
# coloring = ["crimson", "teal"]

###############################################################
###############################################################
# labeling = ["M82/LMC", "MW", "SMC"]

# labeling = ["PIE + CIE", "CIE Only"]

if len(outputs) == 2:
    coloring = ["crimson", "royalblue"]
elif len(outputs) == 3:
    coloring = ["crimson", "teal", "darkgreen"]
else:
    color_map = plt.get_cmap('turbo')
    coloring = color_map(np.linspace(0, 1, len(outputs)))

CC85_PLOTS = True
COOLING = True
LOAD_TESTS = True
METALLICTY_TESTS = False

if COOLING: 
    tcat_labels= [
                    Line2D([0], [0], color='black', linestyle='solid'),        
                    Line2D([0], [0], color='black', linestyle='dashed'),
                    Line2D([0], [0], color='black', linestyle="dotted")
                ]
    tcat_names = ["Total", "$T \leq 1e4.5 \, K$", "CC85"] # , "$5.5e5 \, K \,\, \geq T \,\, > 1e4.5 \, K$ ", "$T > 5e5 \, K$ "]
else:  
    nct_labels = [
                    Line2D([0], [0], color='black', linestyle='solid'),        
                    Line2D([0], [0], color='black', linestyle='dashed'),
                    Line2D([0], [0], color='black', linestyle="dotted")
                ]
    nct_names = ["Simulation", r"$\rm T \leq 1e4.5 K$","CC85"] # , "$5.5e5 \, K \,\, \geq T \,\, > 1e4.5 \, K$ ", "$T > 5e5 \, K$ "]

# Radial Profiles
Bins and plots median radial profiles of gas cells within a 120 degree bicone.
### Aquiring the Data
Calculates the median radial profile, along with the CC85 solution. If cooling is enabled, we also compute the median profile of cells with $T\leq 1e4.5K$<br>

In [ ]:
n_bins = 600
n_bins_cwh = 100
n_snaps = 4

med_number_density = np.zeros(shape=(len(outputs), n_bins))
med_vel = np.zeros(shape=(len(outputs), n_bins))
med_Temp = np.zeros(shape=(len(outputs), n_bins))
med_Press = np.zeros(shape=(len(outputs), n_bins))
med_csound = np.zeros(shape=(len(outputs), n_bins))
med_Mach = np.zeros(shape=(len(outputs), n_bins))
med_entropy = np.zeros(shape=(len(outputs), n_bins))

number_density_84 = np.zeros(shape=(len(outputs),n_bins))
rad_vel_84 = np.zeros(shape=(len(outputs), n_bins))
Temp_84 = np.zeros(shape=(len(outputs), n_bins))
Press_84 = np.zeros(shape=(len(outputs), n_bins))
csound_84 = np.zeros(shape=(len(outputs), n_bins))
Mach_84 = np.zeros(shape=(len(outputs), n_bins))
entropy_84 = np.zeros(shape=(len(outputs), n_bins))

number_density_16 = np.zeros(shape=(len(outputs),n_bins))
rad_vel_16 = np.zeros(shape=(len(outputs), n_bins))
rad_vel_16 = np.zeros(shape=(len(outputs), n_bins))
Temp_16 = np.zeros(shape=(len(outputs), n_bins))
Press_16 = np.zeros(shape=(len(outputs), n_bins))
csound_16 = np.zeros(shape=(len(outputs), n_bins))
Mach_16 = np.zeros(shape=(len(outputs), n_bins))
entropy_16 = np.zeros(shape=(len(outputs), n_bins))

med_number_density_c = np.zeros(shape=(len(outputs), n_bins_cwh))
med_vel_c = np.zeros(shape=(len(outputs), n_bins_cwh))
med_Temp_c = np.zeros(shape=(len(outputs), n_bins_cwh))
med_Press_c = np.zeros(shape=(len(outputs), n_bins_cwh))
med_csound_c = np.zeros(shape=(len(outputs), n_bins_cwh))
med_Mach_c = np.zeros(shape=(len(outputs), n_bins_cwh))

# med_number_density_w = np.zeros(shape=(len(outputs), n_bins_cwh))
# med_vel_w = np.zeros(shape=(len(outputs), n_bins_cwh))
# med_Temp_w = np.zeros(shape=(len(outputs), n_bins_cwh))
# med_Press_w = np.zeros(shape=(len(outputs), n_bins_cwh))
# med_csound_w = np.zeros(shape=(len(outputs), n_bins_cwh))
# med_Mach_w = np.zeros(shape=(len(outputs), n_bins_cwh))

#### ANALYTIC ####
r_an = np.linspace(0.001, 20, 1000)
cc85_number_density = np.zeros(shape=(len(outputs), len(r_an)))
cc85_vel = np.zeros(shape=(len(outputs), len(r_an)))
cc85_Temp = np.zeros(shape=(len(outputs), len(r_an)))
cc85_Press = np.zeros(shape=(len(outputs), len(r_an)))

# take the average over the last two million years in "real" time
for o, output in enumerate(outputs):
    data = {}
    if "cc85" in output and output != "cc85_disk/output_cooling_params/": # since output_cooling_params was ran after the others.
        snap_range = np.arange(101 - int(n_snaps/2), 101) if any(o in output for o in ["large", "small"]) else np.arange(201 - n_snaps, 201)
    else: snap_range = np.arange(101 - n_snaps, 101)
    # elif "small" in output: snap_range = np.arange(50 - n_snaps, 51)
    for i in snap_range:
        filename = output + "snap_%03d.hdf5" % i
        # print(filename)
        with h5py.File(filename,'r') as f:
            for key in f['PartType0']:
                data[key] = f['PartType0'][key][()]
            header = dict(f['Header'].attrs)
            parameters = dict(f['Parameters'].attrs)
        boxsize = parameters["BoxSize"]
        alpha = parameters["E_load"]
        beta = parameters["M_load"]
        coord = data["Coordinates"]
        x_coord = data["Coordinates"][:,0] 
        y_coord = data["Coordinates"][:,1]
        z_coord = data["Coordinates"][:,2]
        density = data["Density"]
        number_density = density*UnitNumberDensity # Units: cm^{-3}

        internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
        masses = data["Masses"] 
        pressures = data["Pressure"] 
        if "cc85" in output: 
            abundance = 1 
        else:
            InitDiskMetallicity = float(parameters["InitDiskMetallicity"])
            Z = InitDiskMetallicity/Z_solar
            abundance = data["ElectronAbundance"]
        temperature = Temp_S(abundance, internal_energy)
        vel_x = data["Velocities"][:,0]
        vel_y = data["Velocities"][:,1] 
        vel_z = data["Velocities"][:,2] 
        vel = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
        E = internal_energy*masses # NOTE: This is the actual internal energy
        R = parameters["injection_radius"]
        sfr = parameters["sfr"]
        rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord -   0.5*boxsize, z_coord - 0.5*boxsize
        radius = np.sqrt(rad_x**2 + rad_y**2 + rad_z**2)
        radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) # Units: km/s

        theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
        angular_region = (np.abs(theta) <= 60) & (radius <= 20) # & (np.abs(rad_z) >= R) #  & (np.abs(rad_z) >= 0.3) # Excludes anything with absolute angles greater than 60 and the injection radius

        # Values to plot -> restricting ourselves to an angular region
        r_ang = radius[angular_region]
        density_cgs = density[angular_region]*UnitDensity_in_cgs
        nd_ang = number_density[angular_region]

        if COOLING: 
            angular_region_cold = (np.abs(theta) <= angle_l) & (radius <= 20) &  (temperature <= 3e4) # & (np.abs(rad_z) >= R) # Excludes anything with absolute angles greater than 60 
            r_ang_c = radius[angular_region_cold]
            nd_ang_c = number_density[angular_region_cold]
            med_nd_cold, rcwh_edge, _ = stats.binned_statistic(r_ang_c, nd_ang_c, bins=n_bins_cwh, statistic="median", range=(0, 20))

        med_nd, r_edge, _ = stats.binned_statistic(r_ang, nd_ang, bins=n_bins, statistic="median", range=(0, 20))
        h1sta_nd, r_edge, _ = stats.binned_statistic(r_ang, nd_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 84), range=(0, 20))
        h1stb_nd, r_edge, _ = stats.binned_statistic(r_ang, nd_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 16), range=(0, 20))

        rv_ang = radial_velocity[angular_region]

        med_rv, r_edge, _ = stats.binned_statistic(r_ang, rv_ang, bins=n_bins, statistic="median", range=(0, 20))

        h1sta_rv, r_edge, _ = stats.binned_statistic(r_ang, rv_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 84), range=(0, 20))
        h1stb_rv, r_edge, _ = stats.binned_statistic(r_ang, rv_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 16), range=(0, 20))
        if COOLING:
            rv_ang_c = radial_velocity[angular_region_cold]
            med_rv_cold, rcwh_edge, _ = stats.binned_statistic(r_ang_c, rv_ang_c, bins=n_bins_cwh, statistic="median", range=(0, 20))

        temp_ang = temperature[angular_region]

        med_T, r_edge, _ = stats.binned_statistic(r_ang, temp_ang, bins=n_bins, statistic="median", range=(0, 20))
        h1sta_T, r_edge, _ = stats.binned_statistic(r_ang, temp_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 84), range=(0, 20))
        h1stb_T, r_edge, _ = stats.binned_statistic(r_ang, temp_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 16), range=(0, 20))
        if COOLING:
            temp_ang_c = temperature[angular_region_cold]
            med_temp_cold, rcwh_edge, _ = stats.binned_statistic(r_ang_c, temp_ang_c, bins=n_bins_cwh, statistic="median", range=(0, 20))

        pressure_cgs = pressures*UnitPressure_in_cgs/kb
        press_ang = pressure_cgs[angular_region]
        med_P, r_edge, _ = stats.binned_statistic(r_ang, press_ang, bins=n_bins, statistic="median", range=(0, 20))
        h1sta_P, r_edge, _ = stats.binned_statistic(r_ang, press_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 84), range=(0, 20))
        h1stb_P, r_edge, _ = stats.binned_statistic(r_ang, press_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 16), range=(0, 20))

        if COOLING:
            press_ang_c = pressure_cgs[angular_region_cold]
            med_P_cold, rcwh_edge, _ = stats.binned_statistic(r_ang_c, press_ang_c, bins=n_bins_cwh, statistic="median", range=(0, 20))

        if COOLING == False:
            lower_bound, upper_bound = midpoint - dx*6, midpoint + eps*6
            z_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) & (x_coord >=lower_bound) & (x_coord <= upper_bound)
            
        # sound_speed = np.sqrt((gamma*kb*temperature)/(mean_molecular_weight(abundance)))  # sqrt((erg/K * K)/grams) = sqrt((cm/s)^2) = cm/s
        # sound_speed /= UnitVelocity_in_cm_per_s # to get in in km/s like radial vleocity 
        # cs_ang = sound_speed[angular_region]
        # cs_ang_c = sound_speed[angular_region_cold]
        # med_cs, r_edge, _ = stats.binned_statistic(r_ang, cs_ang, bins=n_bins, statistic="median", range=(0, 20))
        # h1sta_cs, r_edge, _ = stats.binned_statistic(r_ang, cs_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 84), range=(0, 20))
        # h1stb_cs, r_edge, _ = stats.binned_statistic(r_ang, cs_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 16), range=(0, 20))

        # med_cs_cold, rcwh_edge, _ = stats.binned_statistic(r_ang_c, cs_ang_c, bins=n_bins_cwh, statistic="median", range=(0, 20))

        # mach_number = vel/sound_speed
        # M_ang = mach_number[angular_region]
        # M_ang_c = mach_number[angular_region_cold]
        # med_M, r_edge, _ = stats.binned_statistic(r_ang, M_ang, bins=n_bins, statistic="median", range=(0, 20))
        # h1sta_M, r_edge, _ = stats.binned_statistic(r_ang, M_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 84), range=(0, 20))
        # h1stb_M, r_edge, _ = stats.binned_statistic(r_ang, M_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 16), range=(0, 20))

        # med_M_cold, rcwh_edge, _ = stats.binned_statistic(r_ang_c, M_ang_c, bins=n_bins_cwh, statistic="median", range=(0, 20))

        med_number_density[o] += med_nd
        number_density_84[o] += h1sta_nd
        number_density_16[o] += h1stb_nd

        med_vel[o] += med_rv
        rad_vel_84[o] += h1sta_rv
        rad_vel_16[o] += h1stb_rv

        med_Temp[o] += med_T
        Temp_84[o] += h1sta_T
        Temp_16[o] += h1stb_T

        med_Press[o] += med_P
        Press_84[o] += h1sta_P
        Press_16[o] += h1stb_P

        # med_csound[o] += med_cs
        # csound_84[o] += h1sta_cs
        # csound_16[o] += h1stb_cs
        # med_csound_c[o] += med_cs_cold

        # med_Mach[o] += med_M
        # Mach_84[o] += h1sta_M
        # Mach_16[o] += h1stb_M
        # med_Mach_c[o] += med_M_cold

        r_face = (r_edge[:-1] + r_edge[1:]) / 2
        # rcwh_face = (rcwh_edge[:-1] + rcwh_edge[1:]) / 2

        if COOLING:
            med_Press_c[o] += med_P_cold
            med_Temp_c[o] += med_temp_cold
            med_vel_c[o] += med_rv_cold
            med_number_density_c[o] += med_nd_cold
            rcwh_face = (rcwh_edge[:-1] + rcwh_edge[1:]) / 2

    if LOAD_TESTS: labeling.append(r"$\alpha = %.1f, \beta = %.1f$" % (parameters["E_load"], parameters["M_load"]))
    if METALLICTY_TESTS: labeling.append("Non-Cooling" if "cc85_disk" in output else r"$Z = %0.2f \, Z_\odot$" % Z)
    # labeling.append(r"$\rm R_{inject} = %0.2f$ kpc" % (R))
    if (CC85_PLOTS):
        M_load = parameters["M_load"]
        E_load = parameters["E_load"]
        R = parameters["injection_radius"]
        sfr = parameters["sfr"]

        r_in = r_an[np.where(r_an <= R)]
        r_out = r_an[np.where(r_an > R)]

        s_in_yr = 3.154e+7
        grams_in_M_sun = 1.989e33
        M_dot_wind = sfr*M_load # solar masses per 1 year -> get this in grams per second 
        M_dot_cm = (M_dot_wind*UnitMass_in_g)/s_in_yr # grams/second
        E_dot_wind = E_load*3e41*sfr # this is in ergs/second 

        M_dot_code = M_dot_wind/(UnitMass_in_g/grams_in_M_sun)*(UnitTime_in_s/s_in_yr)
        E_dot_code = E_dot_wind/UnitEnergy_in_cgs*UnitTime_in_s

        M1 = optimize.fsolve(sol_in, x0=np.full(len(r_in), 0.001), args=(r_in, R))
        M2 = optimize.fsolve(sol_out, x0=np.full(len(r_out), 100), args=(r_out, R))
        M = np.concatenate([M1, M2])

        v_an = (M*np.sqrt(E_dot_code/M_dot_code)*(((gamma - 1)*M**2 + 2)/(2*(gamma - 1)))**(-0.5)) # this is in code units
        v_in = v_an[np.where(r_an <= R)]
        v_out = v_an[np.where(r_an > R)]
        cs = np.sqrt( (E_dot_code/M_dot_code)*(((gamma - 1)*M**2 + 2)/(2*(gamma - 1)))**(-1))
        cs_cm = cs*(UnitVelocity_in_cm_per_s) 

        rho_in = M_dot_code/(4*np.pi*v_in)*(r_in/R**3)*UnitDensity_in_cgs
        rho_out = M_dot_code/(4*np.pi*v_out)*1/r_out**2*UnitDensity_in_cgs


        rho_an = np.concatenate([rho_in, rho_out])
        rho_n = np.concatenate([rho_in, rho_out])/PROTON_MASS_GRAMS # rho/(proton mass)

        pressure_an = ((rho_an*cs_cm**2)/gamma)/kb # -> (g/cm^3* cm^2/s^2) -> p/kb 

        temp_an = pressure_an/(rho_n)*(mean_molecular_weight(1)/PROTON_MASS_GRAMS) # keep the mean molecular mass the same. 

        cc85_number_density[o] += rho_n
        cc85_Temp[o] += temp_an
        cc85_vel[o] += v_an
        cc85_Press[o] += pressure_an

denoms = np.array([int(n_snaps/2)  if ("cc85_disk" in o) and (("large" in o) or ("small" in o)) else n_snaps for o in outputs])[:, None]

med_number_density /= denoms
number_density_84 /= denoms
number_density_16 /= denoms
med_number_density_c /= denoms

med_vel /= denoms
rad_vel_84 /= denoms
rad_vel_16 /= denoms
med_vel_c /= denoms

med_Temp /= denoms
Temp_84 /= denoms
Temp_16 /= denoms
med_Temp_c /= denoms

med_Press /= denoms
Press_84 /= denoms
Press_16 /= denoms
med_Press_c /= denoms

# med_csound /= n_snaps
# csound_84 /= n_snaps
# csound_16 /= n_snaps
# med_csound_c /= n_snaps

# med_Mach /= n_snaps
# Mach_84 /= n_snaps
# Mach_16 /= n_snaps

# med_Mach_c /= n_snaps

### Velocity, Temperature, Density, Pressure

In [ ]:
print(labeling)
fig = plt.figure(figsize=(10, 8))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

for i in np.arange(len(outputs)):

    ax1.semilogy(r_face, med_number_density[i], label=labeling[i], linestyle="solid", color=coloring[i])
    ax1.fill_between(r_face,  number_density_16[i], number_density_84[i], color=coloring[i], alpha=0.1)
    if COOLING: 
        ax1.semilogy(rcwh_face, med_number_density_c[i], linestyle="dashed", color=coloring[i])
        l2 = ax1.legend(tcat_labels, tcat_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize=13)
    else:
        l2 = ax1.legend(nct_labels, nct_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize=13)

    l1 = ax1.legend(loc="upper right", fontsize=13)
    ax1.add_artist(l1)
    ax1.add_artist(l2)

    ax2.plot(r_face, med_vel[i], label=labeling[i], linestyle="solid", color=coloring[i])
    ax2.fill_between(r_face, rad_vel_16[i], rad_vel_84[i], color=coloring[i], alpha=0.1)
    if COOLING: 
        ax2.plot(rcwh_face, med_vel_c[i], linestyle="dashed", color=coloring[i])
        l2 = ax2.legend(tcat_labels, tcat_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize=13)
    else:
        l2 = ax2.legend(nct_labels, nct_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize=13)
      
    ax2.set_yticks([200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400])
    l1 = ax2.legend(loc="upper right", fontsize=13)
    ax2.add_artist(l1)
    ax2.add_artist(l2)

    ax3.semilogy(r_face, med_Temp[i], label=labeling[i], linestyle="solid",  color=coloring[i])
    ax3.fill_between(r_face, Temp_16[i], Temp_84[i], color=coloring[i], alpha=0.1)
    if COOLING: 
        ax3.semilogy(rcwh_face, med_Temp_c[i], linestyle="dashed", color=coloring[i])
        l2 = ax3.legend(tcat_labels, tcat_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize=13)
    else:
        l2 = ax3.legend(nct_labels, nct_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize=13)
      
    l1 = ax3.legend(loc="upper right", fontsize=13)
    ax3.add_artist(l1)
    ax3.add_artist(l2)

    ax4.semilogy(r_face, med_Press[i], label=labeling[i], linestyle="solid", color=coloring[i])
    ax4.fill_between(r_face, Press_16[i], Press_84[i], color=coloring[i], alpha=0.1)
    if COOLING: 
        ax4.semilogy(rcwh_face, med_Press_c[i], linestyle="dashed", color=coloring[i])
        l2 = ax4.legend(tcat_labels, tcat_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize=13)
    else:
        l2 = ax4.legend(nct_labels, nct_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize=13)
      
    l1 = ax4.legend(loc="upper right", fontsize=13)
    ax4.add_artist(l1)
    ax4.add_artist(l2)

    if CC85_PLOTS:
        if LOAD_TESTS == False and i == 0:
            ax1.semilogy(r_an, cc85_number_density[0], linestyle="dotted", color="black")
            ax2.plot(r_an, cc85_vel[0],  linestyle="dotted", color="black")
            ax3.semilogy(r_an, cc85_Temp[0], linestyle="dotted",  color="black")
            ax4.semilogy(r_an, cc85_Press[0], linestyle="dotted", color="black")
        else:
            ax1.semilogy(r_an, cc85_number_density[i], linestyle="dotted", color=coloring[i])
            print(cc85_number_density[i])
            ax2.plot(r_an, cc85_vel[i],  linestyle="dotted", color=coloring[i])
            ax3.semilogy(r_an, cc85_Temp[i], linestyle="dotted",  color=coloring[i])
            ax4.semilogy(r_an, cc85_Press[i], linestyle="dotted", color=coloring[i])

ax1.set(xlim=(0, 20), ylim=(1e-5, 100)) 
ax2.set(xlim=(0, 20), ylim=(100, 2000))
ax3.set(xlim=(0, 20), ylim=(1e3, 1e9))
ax4.set(xlim=(0, 20), ylim=(1, 1e8))

ax1.set_ylabel(ylabel=(r"Density [$\rm cm^{-3}$]"), fontsize=14)
ax2.set_ylabel(ylabel=("Radial Velocity [km/s]"), fontsize=14)
ax3.set_ylabel("Temperature [K]", fontsize=14)
ax4.set_ylabel(r"Pressure [$\rm K \, cm^{-3}$]", fontsize=14)

ax3.set_xlabel("Radius [kpc]", fontsize=14)
ax4.set_xlabel("Radius [kpc]", fontsize=14)

ax1.tick_params(axis='both', which='major', labelsize=12)
ax2.tick_params(axis='both', which='major', labelsize=12)
ax3.tick_params(axis='both', which='major', labelsize=12)
ax4.tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout(w_pad=0.0, h_pad=0)

ax4.set_yticks([10**x for x in np.arange(0,9)])

plt.savefig("./nd_vr_T_P_profile.pdf", bbox_inches='tight')
plt.show()

#### Temperature radial profiles vs CGOLs
Compares the radial profiles of the temperature with that of the central feedback model of Schneider et. al (https://arxiv.org/abs/1803.01005)

In [ ]:
import csv

def calc_cool_radius(alpha, beta, R_inject, opening_angle, Z, M_sfr, mu):
    R_03 = R_inject/0.3
    xi = Z/Z_solar
    M_stardot = M_sfr/10
    Omega_4pi = opening_angle/(4*np.pi)

    return 4 * ((pow(alpha,2.13))/pow(beta, 2.92)) * pow(mu, 2.13) * pow(R_03, 1.79) * (Omega_4pi/(xi*M_stardot))**0.789

c_radius = calc_cool_radius(E_load, M_load, R, 4*np.pi, 0.02, sfr, 0.6)

with open('./Temperature_CGOLS_model_B.csv', newline='') as f:
    b_reader = csv.reader(f)
    model_b_T = np.array(list(b_reader), dtype="float")

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111)
ax.semilogy(r_face, med_Temp[1], color="crimson", label=labeling[1])
ax.semilogy(r_face, med_Temp[2], color="blue", label=labeling[2])
ax.semilogy(model_b_T[:,0], model_b_T[:,1], color="black", label=r"CGOLS Model B - 25 Myr (2018)")
ax.legend(loc="upper right", fontsize=14)
ax.axvline(c_radius, color="black", linestyle="dashed")
ax.set_xlim(0.3,10)
ax.set_yticks([1e3*(10**x) for x in np.arange(1,5)])
ax.set_yticklabels(np.log10([1e3*(10**x) for x in np.arange(1,5)]), fontsize=10)
ax.set_xticklabels([0.3, 2, 4, 5, 6, 8], fontsize=10)
ax.set_ylabel(r"Temperature [$\rm log_{10}(K)$]", fontsize=14)
ax.set_xlabel("Radius [kpc]", fontsize=14)
ax.tick_params(axis='both', which='major', labelsize="large")

plt.savefig("T_CGOLS_comparision.pdf", bbox_inches='tight', dpi=150)


### Entropy
Note that in the thesis, we did not examine the entropy in detail.

In [ ]:

color_map = plt.get_cmap('turbo')
coloring = color_map(np.linspace(0, 1, len(outputs)))

theta = 60
n_snaps = 4

med_entropy = np.zeros(shape=(len(outputs), n_bins))
entropy_84 = np.zeros(shape=(len(outputs), n_bins))
entropy_16 = np.zeros(shape=(len(outputs), n_bins))

med_entropy_cold = np.zeros(shape=(len(outputs), int(n_bins/2)))
entropy_84_cold = np.zeros(shape=(len(outputs), int(n_bins/2)))
entropy_16_cold = np.zeros(shape=(len(outputs), int(n_bins/2)))

for o, output in enumerate(outputs):
    print(output)
    data = {}
    for i in np.arange(101 - n_snaps, 101):
        filename = output + "snap_%03d.hdf5" % i
        print(filename)
        with h5py.File(filename,'r') as f:
            for key in f['PartType0']:
                data[key] = f['PartType0'][key][()]
            header = dict(f['Header'].attrs)
            parameters = dict(f['Parameters'].attrs)
        boxsize = parameters["BoxSize"]
        alpha = parameters["E_load"]
        beta = parameters["M_load"]
        coord = data["Coordinates"]
        x_coord = data["Coordinates"][:,0] 
        y_coord = data["Coordinates"][:,1]
        z_coord = data["Coordinates"][:,2]
        density = data["Density"]
        number_density = density*UnitNumberDensity # Units: cm^{-3}

        internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
        masses = data["Masses"] 
        pressures = data["Pressure"] 
        abundance = data["ElectronAbundance"]
        temperature = Temp_S(abundance, internal_energy)

        vel_x = data["Velocities"][:,0]
        vel_y = data["Velocities"][:,1] 
        vel_z = data["Velocities"][:,2] 
        vel = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
        E = internal_energy*masses # NOTE: This is the actual internal energy
        R = parameters["injection_radius"]
        sfr = parameters["sfr"]
        rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
        radius = np.sqrt(rad_x**2 + rad_y**2 + rad_z**2)
        radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) # Units: km/s

        theta = np.arccos(np.abs(rad_z)/(radius + dx/1e6))*180/np.pi 

        angular_region = (np.abs(theta) <= angle_l) & (radius <= 8) & ((np.abs(rad_z) >= R)) # Excludes anything with absolute angles greater than 60 

        # Values to plot -> restricting ourselves to an angular region
        r_ang = radius[angular_region]
        density_cgs = density*UnitDensity_in_cgs 
        nd_ang = number_density[angular_region]
        pressure_cgs = pressures*UnitPressure_in_cgs/kb # p =  n kb T -> n T = p/kb, n = p/(kb T)
        press_ang = pressure_cgs[angular_region]


        a_cold = angular_region & (temperature <= 3e4) # Excludes anything with absolute angles greater than 60 
        r_cold = radius[a_cold]

        # Dullemond's notes: 1.12 - K = P/rho^-gamma, S = M*s 
        # where s = k/(mu(gamma -1))*ln(K) -> eqs 1.22, and mu is the med weight of the gas particles (in gram) 
        # entropy = kb/(med_molecular_weight(1) * (gamma - 1)) * np.log(pressure_cgs/(pow(density_cgs, 5/3))) # specific entropy in (ergs/Kelvin/grams)
        # specific_entropy = kb/(mean_molecular_weight(abundance) * (gamma - 1)) * np.log(pressure_cgs/(pow(density_cgs, 5/3))) # specific entropy from equation 1.22
        # print(specific_entropy*masses)
        # K_ang = specific_entropy[angular_region]

        # CGOLS : K propto P * rho^{-gamma} though in figures 6 and 7, K was ploted as K propto T n^{-2/3} 
        ## P/kb/(n)^(5/3) = nT/(n^(5/3)) = T n^{-2/3} so just keep what I currently have.
        entropy = pressure_cgs * pow(density_cgs, -5/3)
        K_ang = entropy[angular_region]
        med_K, r_edge, _ = stats.binned_statistic(r_ang, K_ang, bins=n_bins, statistic="median", range=(0.5, 10))
        h1sta_K, r_edge, _ = stats.binned_statistic(r_ang, K_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 84), range=(0.5, 10))
        h1stb_K, r_edge, _ = stats.binned_statistic(r_ang, K_ang, bins=n_bins, statistic=lambda x: np.percentile(x, 16), range=(0.5, 10))

        med_entropy[o] += med_K
        entropy_84[o] += h1sta_K
        entropy_16[o] += h1stb_K


        K_cold = entropy[a_cold]
        med_K_cold, rc_edge, _ = stats.binned_statistic(r_cold, K_cold, bins=int(n_bins/2), statistic="median", range=(0.5, 10))
        h1sta_K_cold, rc_edge, _ = stats.binned_statistic(r_cold, K_cold, bins=int(n_bins/2),statistic=lambda x: np.percentile(x, 84), range=(0.5, 10))
        h1stb_K_cold, rc_edge, _ = stats.binned_statistic(r_cold, K_cold, bins=int(n_bins/2), statistic=lambda x: np.percentile(x, 16), range=(0.5, 10))

        med_entropy_cold[o] += med_K_cold
        entropy_84_cold[o] += h1sta_K_cold
        entropy_16_cold[o] += h1stb_K_cold

        r_face = (r_edge[:-1] + r_edge[1:]) / 2
        rc_face = (rc_edge[:-1] + rc_edge[1:]) / 2

med_entropy /= n_snaps
entropy_84 /= n_snaps
entropy_16 /= n_snaps

med_entropy_cold /= n_snaps
entropy_84_cold /= n_snaps
entropy_16_cold /= n_snaps

fig = plt.figure(figsize=(6, 6))
ax1 = fig.add_subplot(111)
for i in np.arange(len(outputs)):
    ax1.semilogy(r_face, med_entropy[i], label=labeling[i], linestyle="solid", color=colors[i])
    ax1.fill_between(r_face,  entropy_16[i], entropy_84[i], color=colors[i], alpha=0.1)
    ax1.semilogy(rc_face, med_entropy_cold[i], linestyle="dotted", color=colors[i])

ax1.set(xlabel="Radius [kpc]", xlim=(0.5,8), ylabel=("Entropy [$K \, \, cm^{2}$]"), ylim=(1e44, 1e49))
ax1.legend(loc="upper left")
plt.savefig("./entropy_plots.pdf", bbox_inches='tight', dpi=150)
plt.show()

# Cooling Related Plots

### Temperature
#### Temperaure Profiles and Edge-on Slices - Version 1.
Plots the temperature profiles from Cell 2 and the edge-on temperature slice for 4 snapshots listed in `files`. Was used for the MFP examination.

In [ ]:
from utilities import plot_edge
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

lower_bound, upper_bound = midpoint - dx*3, midpoint + dx*3
######### SIMULATION DATA #########
data = {}
files = ["./outflow_cooling/output_PIE_fid/snap_100.hdf5", "./outflow_cooling/output_high_alpha/snap_100.hdf5", "./outflow_cooling/output_high_beta/snap_100.hdf5",
         "./outflow_cooling/output_high_alpha_high_beta/snap_100.hdf5"]

fig = plt.figure(figsize=(16,8))
fig.set_rasterized(True)    
gs = GridSpec(2, 3, width_ratios=[1.5, 1, 1], height_ratios=[1, 1], wspace=0.185, hspace=0.2)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 1])
ax5 = fig.add_subplot(gs[1, 2])

axs = [ax2, ax3, ax4, ax5]

color_map = plt.get_cmap('turbo')
coloring = color_map(np.linspace(0, 1, len(files)) )

deviation = 10
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2 + deviation  # boundary of histogram - lower bound


for i, file in enumerate(files): # select the snapshot range to go through
    with h5py.File(file,'r') as f:
        for key in f['PartType0']:
            data[key] = f['PartType0'][key][()]
        header = dict(f['Header'].attrs)
        parameters = dict(f['Parameters'].attrs)
    R = parameters["injection_radius"]
    E_load = parameters["E_load"]
    M_laod = parameters["M_load"]

    coord = data["Coordinates"]
    x_coord = data["Coordinates"][:,0] 
    y_coord = data["Coordinates"][:,1]
    z_coord = data["Coordinates"][:,2]
    density = data["Density"]
    internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
    masses = data["Masses"] 
    vel_x = data["Velocities"][:,0]
    vel_y = data["Velocities"][:,1] 
    vel_z = data["Velocities"][:,2] 
    volume = masses/density
    number_density = density*UnitNumberDensity
    abundance = data["ElectronAbundance"]
    temperature = Temp_S(abundance, internal_energy)
    t = header["Time"]
    times = t*1000
    ''' Get the radial distance of the box'''
    rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
    radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)

    edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) # & (radius <= center_boxsize/2*np.sqrt(3))
    theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
    angular_region = (np.abs(theta) <= angle_l) & (radius <= 20) & ((np.abs(rad_z) >= R))# Excludes anything with absolute angles greater than 60 

    volume = masses/density

    plot_edge(axs[i], coord[edge_mask], temperature[edge_mask], n_bins*2, midpoint, boxsize, 1e3, 1e7, histb_l, histb_h, log=True)
    T_mesh = axs[i].collections[0]
    T_mesh.set_cmap("plasma")
    cbar = plt.colorbar(T_mesh, ax = axs[i], label='Temperature [log(K)]')
    labels = [1*(10**(x)) for x in range(3,8)]
    cbar.set_ticks(labels)
    cbar.set_ticklabels([int(np.log10(label)) for label in labels]) 
    background_rect = patches.Rectangle((0, 0.80), width=1, height=0.2, color='black', alpha=0.25, transform=axs[i].transAxes, fill=True)
    axs[i].add_patch(background_rect)

    axs[i].text(0.03, 0.91, "t = %0.1f Myr" % times, transform=axs[i].transAxes, color="white", fontsize="large")
    axs[i].text(0.03, 0.84, r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]), transform=axs[i].transAxes, color="white", fontsize="large")
    labeling.append(r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]))

    axs[i].set_xticks([40, 45, 50, 55, 60])
    axs[i].set_yticks([40, 45, 50, 55, 60])

for i in np.arange(len(files)):
    ax1.semilogy(r_face, med_Temp[i], label=labeling[i], linestyle="solid",  color=coloring[i])
    ax1.fill_between(r_face, Temp_16[i], Temp_84[i], color=coloring[i], alpha=0.1)
    ax1.set(xlim=(0.5, 20), ylim=(1e3, 1e8))
    ax1.set_ylabel("Temperature [K]", fontsize=12) 
    if CC85_PLOTS: ax1.semilogy(r_an, cc85_Temp[i],linestyle="dotted", color=coloring[i])


pos = ax1.get_position()  # get current position [left, bottom, width, height]
ax1.set_position([
    pos.x0,        # keep left position
    pos.ymax/3,        
    pos.width*1, 
    pos.height*0.66 
])

l1 = ax1.legend(loc="upper right", ncol=2, fontsize="large", title="Median Temperatures(48 - 50 Myr) ",  title_fontsize="large")

T_labels = [
                Line2D([0], [0], color='black', linestyle='solid'),        
                Line2D([0], [0], color='black', linestyle="dotted")
            ]
T_names = ["Simulation", "CC85"] # , "$5.5e5 \, K \,\, \geq T \,\, > 1e4.5 \, K$ ", "$T > 5e5 \, K$ "]
l2 = ax1.legend(T_labels, T_names, loc ="lower right", fontsize="large")

ax1.add_artist(l1)
ax1.add_artist(l2)

ax1.set(xlim=(0, 20), ylim=(1e3, 1e9))

ax1.text(0.58, 0.70, r"Simulation Parameters", transform=ax1.transAxes, color="black",fontsize="large")
ax1.text(0.58, 0.65, r"$\rm \dot{M}_{SFR} = 20 M_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.58, 0.60, r"$\rm Z = 4 Z_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.58, 0.55, r"$\rm R_{inject}$ = 300 pc", transform=ax1.transAxes, color="black",  fontsize="large")

ax1.xaxis.set_tick_params(labelsize="large")
ax1.yaxis.set_tick_params(labelsize="large")

ax1.set_xlabel("Radius [kpc]", fontsize=13)
ax1.set_ylabel("Temperature [K]", fontsize=13)

plt.tight_layout(w_pad=0.0)
plt.savefig("temp_comp_all_configs_v1.pdf", dpi=150, bbox_inches='tight', facecolor='white', transparent=False)

#### Temperature Profiles and Edge-On Slices - Version 2
Plots the temperature profiles from Cell 2 and the edge-on temperature slice for a given snapshot. Was the version that was included in the thesis.

In [ ]:
from utilities import plot_edge
import matplotlib.patches as patches

lower_bound, upper_bound = midpoint - dx*4, midpoint + dx*4

######### SIMULATION DATA #########
data = {}
snapshots_voronoi = "./outflow_cooling/output_high_beta/snap_100.hdf5"
fig = plt.figure(figsize=(11, 5))
fig.set_rasterized(True)    
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

deviation = 10
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2 + deviation  # boundary of histogram - lower bound

with h5py.File(snapshots_voronoi,'r') as f:
    for key in f['PartType0']:
        data[key] = f['PartType0'][key][()]
    header = dict(f['Header'].attrs)
    parameters = dict(f['Parameters'].attrs)
  
coord = data["Coordinates"]
x_coord = data["Coordinates"][:,0] 
y_coord = data["Coordinates"][:,1]
z_coord = data["Coordinates"][:,2]
density = data["Density"]
internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
masses = data["Masses"] 
vel_x = data["Velocities"][:,0]
vel_y = data["Velocities"][:,1] 
vel_z = data["Velocities"][:,2] 
volume = masses/density
number_density = density*UnitNumberDensity
abundance = data["ElectronAbundance"]
temperature = Temp_S(abundance, internal_energy)
t = header["Time"]
times = t*1000
''' Get the radial distance of the box'''
rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)

edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) # & (radius <= center_boxsize/2*np.sqrt(3))
theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
angular_region = (np.abs(theta) <= angle_l) & (radius <= 20) & ((np.abs(rad_z) >= R))# Excludes anything with absolute angles greater than 60 

volume = masses/density

#### Velocities - for the center disk plane face####
### PLOTS ###
plot_edge(ax2, coord[edge_mask], temperature[edge_mask], n_bins*2, midpoint, boxsize, 1e3, 1e7, histb_l, histb_h, log=True)

T_mesh = ax2.collections[0]
T_mesh.set_cmap("inferno")
ax2.set_ylabel("Z [kpc]", fontsize=14)
ax2.set_xlabel("X [kpc]", fontsize=14)

background_rect = patches.Rectangle((0, 0.88), width=1, height=0.12, color='black', alpha=0.25, transform=ax2.transAxes, fill=True)
ax2.add_patch(background_rect)

ax2.text(0.03, 0.95, "t = %0.1f Myr" % times, transform=ax2.transAxes, color="white", fontsize="large")
ax2.text(0.03, 0.90, r"$\beta = %.1f, \alpha = %.1f$, $\rm Z = 4 Z_\odot$, CIE" % (parameters["M_load"], parameters["E_load"]), transform=ax2.transAxes, color="white", fontsize="large")

ax2.set_yticks([40 + 5*x for x in np.arange(0,5)])
ax2.set_xticks([40 + 5*x for x in np.arange(0,5)])

for i in np.arange(len(outputs)):
    ax1.semilogy(r_face, med_Temp[i], label=labeling[i], linestyle="solid",  color=coloring[i])
    ax1.fill_between(r_face, Temp_16[i], Temp_84[i], color=coloring[i], alpha=0.3)

    if CC85_PLOTS: 
        ax1.semilogy(r_an, cc85_Temp[i],linestyle="dotted", color=coloring[i])

ax1.set(xlim=(0.0, 20), ylim=(1e3, 1e8))

labels = [1*(10**(x)) for x in range(3,8)]
cbar.set_ticks(labels)
cbar.set_ticklabels([int(np.log10(label)) for label in labels]) 
cbar = plt.colorbar(T_mesh, ax = ax2)
cbar.set_label(r'Temperature [$\rm log_{10}(K)$]',fontsize=14)
cbar.ax.tick_params(labelsize=12)

l1 = ax1.legend(loc="upper right", ncol=2, fontsize="large", title="Median Temperatures (48 - 50 Myr)",  title_fontsize="large")

T_labels = [
                Line2D([0], [0], color='black', linestyle='solid'),        
                # Line2D([0], [0], color='black', linestyle='dashed'),
                Line2D([0], [0], color='black', linestyle="dotted")
            ]
T_names = ["Simulation", "CC85"] # , "$5.5e5 \, K \,\, \geq T \,\, > 1e4.5 \, K$ ", "$T > 5e5 \, K$ "]
l2 = ax1.legend(T_labels, T_names, loc ="lower right", fontsize=12)

ax1.add_artist(l1)
ax1.add_artist(l2)

ax1.text(0.54, 0.69, r"Simulation Parameters", transform=ax1.transAxes, color="black",fontsize="large")
ax1.text(0.54, 0.64, r"$\rm \dot{M}_{SFR} = 20 M_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.54, 0.59, r"$\rm Z = 4 Z_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.54, 0.54, r"$\rm R_{inject}$ = 300 pc", transform=ax1.transAxes, color="black",  fontsize="large")

ax1.xaxis.set_tick_params(labelsize=12)
ax1.yaxis.set_tick_params(labelsize=12)

ax2.xaxis.set_tick_params(labelsize=12)
ax2.yaxis.set_tick_params(labelsize=12)

ax1.set_xlabel("Radius [kpc]", fontsize=14)
ax1.set_ylabel(r"Temperature [K]", fontsize=14)
plt.tight_layout(w_pad=0.0)
plt.savefig("temp_comp_load_params.pdf", bbox_inches='tight', facecolor='white', transparent=False)

### Density 


#### Density Profiles and Edge-On Slices - Version 1
Plots the density profiles from Cell 2 and the edge-on density slice for the 4 snapshots listed in `files`. Was not used.

In [ ]:
from utilities import plot_edge
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

lower_bound, upper_bound = midpoint - dx*4, midpoint + dx*4
######### SIMULATION DATA #########

data = {}
files = ["./outflow_cooling/output_PIE_fid/snap_100.hdf5", "./outflow_cooling/output_high_alpha/snap_100.hdf5", "./outflow_cooling/output_high_beta/snap_100.hdf5",
         "./outflow_cooling/output_high_alpha_high_beta/snap_100.hdf5"]

fig = plt.figure(figsize=(15, 9))
fig.set_rasterized(True)    
gs = GridSpec(2, 3, width_ratios=[1.5, 1, 1], height_ratios=[1, 1], wspace=0.01, hspace=0.01)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 1])
ax5 = fig.add_subplot(gs[1, 2])

axs = [ax2, ax3, ax4, ax5]

color_map = plt.get_cmap('turbo')
coloring = color_map(np.linspace(0, 1, len(files)) )

deviation = 10
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2 + deviation  # boundary of histogram - lower bound


for i, file in enumerate(files): # select the snapshot range to go through
    with h5py.File(file,'r') as f:
        for key in f['PartType0']:
            data[key] = f['PartType0'][key][()]
        header = dict(f['Header'].attrs)
        parameters = dict(f['Parameters'].attrs)
    R = parameters["injection_radius"]
    E_load = parameters["E_load"]
    M_laod = parameters["M_load"]

    coord = data["Coordinates"]
    x_coord = data["Coordinates"][:,0] 
    y_coord = data["Coordinates"][:,1]
    z_coord = data["Coordinates"][:,2]
    density = data["Density"]
    internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
    masses = data["Masses"] 
    vel_x = data["Velocities"][:,0]
    vel_y = data["Velocities"][:,1] 
    vel_z = data["Velocities"][:,2] 
    volume = masses/density
    number_density = density*UnitNumberDensity
    abundance = data["ElectronAbundance"]
    temperature = Temp_S(abundance, internal_energy)
    t = header["Time"]
    times = t*1000
    ''' Get the radial distance of the box'''
    rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
    radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)

    edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) # & (radius <= center_boxsize/2*np.sqrt(3))
    theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
    angular_region = (np.abs(theta) <= angle_l) & (radius <= 20) & ((np.abs(rad_z) >= R))# Excludes anything with absolute angles greater than 60 

    volume = masses/density

    ### PLOTS ###

    plot_edge(axs[i], coord[edge_mask], number_density[edge_mask], n_bins*2, midpoint, boxsize, 1e-4, 10, histb_l, histb_h, log=True)
    n_mesh = axs[i].collections[0]
    n_mesh.set_cmap("mako")
    if i % 2 == 0: 
        axs[i].set_ylabel("Z [kpc]", fontsize="medium")
        axs[i].set_yticks([40 + 5*x for x in np.arange(0,4)])
    else: 
        axs[i].set_ylabel("")
        plt.setp(axs[i].get_yticklabels(), visible=False)
    
    if i == 2 or i == 3: 
        axs[i].set_xlabel("X [kpc]", fontsize="medium")
        axs[i].set_xticks([40 + 5*x for x in np.arange(0,4)])
    else: 
        axs[i].set_xlabel("")
        plt.setp(axs[i].get_xticklabels(), visible=False)

    background_rect = patches.Rectangle((0, 0.80), width=1, height=0.2, color='black', alpha=0.25, transform=axs[i].transAxes, fill=True)
    axs[i].add_patch(background_rect)

    axs[i].text(0.03, 0.91, "t = %0.1f Myr" % times, transform=axs[i].transAxes, color="white", fontsize="large")
    axs[i].text(0.03, 0.84, r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]), transform=axs[i].transAxes, color="white", fontsize="large")
    labeling.append(r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]))

axs[0].set_yticks([40 + 5*x for x in np.arange(0,5)])
axs[-1].set_xticks([40 + 5*x for x in np.arange(0,5)])

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.905, 0.125, 0.012, 0.755])
ncbar = plt.colorbar(n_mesh, cax = cbar_ax, label=r'Density [log($\rm cm^{-3}$)]')
labels = [1*(10**(x)) for x in range(-4,2)]
ncbar.set_ticks(labels)
ncbar.set_ticklabels([int(np.log10(label)) for label in labels]) 


for i in np.arange(len(files)):
    ax1.semilogy(r_face, med_number_density[i], label=labeling[i], linestyle="solid", color=coloring[i])
    ax1.fill_between(r_face,  number_density_16[i], number_density_84[i], color=coloring[i], alpha=0.1)
    ax1.semilogy(rcwh_face, med_number_density_c[i], linestyle="dashed", color=coloring[i])
    ax1.set(xlim=(0, 20), ylim=(1e-4, 100))
    ax1.set_ylabel(ylabel=(r"Density [$\rm cm^{-3}$]"), fontsize="large")
    if CC85_PLOTS: ax1.semilogy(r_an, cc85_number_density[i],linestyle="dotted", color=coloring[i])

pos = ax1.get_position()  # get current position [left, bottom, width, height]
ax1.set_position([
    pos.x0,        # keep left position
    pos.ymax/3,        
    pos.width*0.86, 
    pos.height*0.55 
])

l1 = ax1.legend(loc="upper right", ncol=2, fontsize="large", title="Median Densities(48 - 50 Myr) ",  title_fontsize="large")
l2 = ax1.legend(tcat_labels, tcat_names, loc ="lower left", fontsize="large")

ax1.add_artist(l1)
ax1.add_artist(l2)

ax1.text(0.54, 0.69, r"Simulation Parameters", transform=ax1.transAxes, color="black",fontsize="large")
ax1.text(0.54, 0.64, r"$\rm \dot{M}_{SFR} = 20 M_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.54, 0.59, r"$\rm Z = 4 Z_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.54, 0.54, r"$\rm R_{inject}$ = 300 pc", transform=ax1.transAxes, color="black",  fontsize="large")

ax1.xaxis.set_tick_params(labelsize="medium")
ax1.yaxis.set_tick_params(labelsize="medium")

ax1.set_xlabel("Radius [kpc]", fontsize="large")
ax1.set_ylabel(r"Density [$cm^{-3}$]", fontsize="large")

plt.savefig("density_comp_load_params_v1.pdf", dpi=150, bbox_inches='tight', facecolor='white', transparent=False)


#### #### Density Profiles and Edge-On Slices - Version 2
Plots the density profiles from Cell 2 and the edge-on density slice for a given snapshot. Was the version that was used for the thesis.

In [ ]:
from utilities import plot_edge
import matplotlib.patches as patches

lower_bound, upper_bound = midpoint - dx*4, midpoint + dx*4

######### SIMULATION DATA #########
data = {}
snapshots_voronoi = "./outflow_cooling/output_high_beta/snap_100.hdf5"
fig = plt.figure(figsize=(11, 5))
fig.set_rasterized(True)    
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

deviation = 20
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2 + deviation  # boundary of histogram - lower bound

with h5py.File(snapshots_voronoi,'r') as f:
    for key in f['PartType0']:
        data[key] = f['PartType0'][key][()]
    header = dict(f['Header'].attrs)
    parameters = dict(f['Parameters'].attrs)

R = parameters["injection_radius"]
E_load = parameters["E_load"]
M_load = parameters["M_load"]

coord = data["Coordinates"]
x_coord = data["Coordinates"][:,0] 
y_coord = data["Coordinates"][:,1]
z_coord = data["Coordinates"][:,2]
density = data["Density"]
internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
masses = data["Masses"] 
vel_x = data["Velocities"][:,0]
vel_y = data["Velocities"][:,1] 
vel_z = data["Velocities"][:,2] 
volume = masses/density
number_density = density*UnitNumberDensity
abundance = data["ElectronAbundance"]
temperature = Temp_S(abundance, internal_energy)
t = header["Time"]
times = t*1000
''' Get the radial distance of the box'''
rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)

edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) # & (radius <= center_boxsize/2*np.sqrt(3))
theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
angular_region = (np.abs(theta) <= angle_l) & (radius <= 20) & ((np.abs(rad_z) >= R))# Excludes anything with absolute angles greater than 60 

volume = masses/density
### PLOTS ###

plot_edge(ax2, coord[edge_mask], number_density[edge_mask], n_bins*2, midpoint, boxsize, 1e-5, 10, histb_l, histb_h, log=True)
n_mesh = ax2.collections[0]
n_mesh.set_cmap("mako")
ax2.set_ylabel("Z [kpc]", fontsize=14)
ax2.set_xlabel("X [kpc]", fontsize=14)

background_rect = patches.Rectangle((0, 0.88), width=1, height=0.12, color='black', alpha=0.25, transform=ax2.transAxes, fill=True)
ax2.add_patch(background_rect)

ax2.text(0.03, 0.95, "t = %0.1f Myr" % times, transform=ax2.transAxes, color="white", fontsize="large")
ax2.text(0.03, 0.90, r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]), transform=ax2.transAxes, color="white", fontsize="large")
labeling.append(r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]))

ax2.set_yticks([30 + 10*x for x in np.arange(0,5)])
ax2.set_xticks([30 + 10*x for x in np.arange(0,5)])

for i in np.arange(len(outputs)):
    ax1.semilogy(r_face, med_number_density[i], label=labeling[i], linestyle="solid", color=coloring[i])
    ax1.fill_between(r_face,  number_density_16[i], number_density_84[i], color=coloring[i], alpha=0.3)
    ax1.semilogy(rcwh_face, med_number_density_c[i], linestyle="dashed", color=coloring[i])
    ax1.set(xlim=(0.0, 20), ylim=(1e-5, 1e2))
    ax1.set_ylabel(r"Density [$\rm cm^{-3}$]", fontsize="medium") 

    if CC85_PLOTS: ax1.semilogy(r_an, cc85_number_density[i],linestyle="dotted", color=coloring[i])

cbar = plt.colorbar(n_mesh, ax = ax2)
cbar.set_label(r'Density [$\rm log_{10}(cm^{-3})$]', fontsize=14)
labels = [1*(10**(x)) for x in range(-5,10)]
cbar.set_ticks(labels)
cbar.set_ticklabels([int(np.log10(label)) for label in labels]) 
cbar.ax.tick_params(labelsize=12)

l1 = ax1.legend(loc="upper right", ncol=2, fontsize=13, title="Median Density (48 - 50 Myr) ",  title_fontsize=13)
l2 = ax1.legend(tcat_labels, tcat_names, loc ="lower left", fontsize=13)
ax1.add_artist(l1)
ax1.add_artist(l2)

ax1.text(0.54, 0.69, r"Simulation Parameters", transform=ax1.transAxes, color="black",fontsize="large")
ax1.text(0.54, 0.64, r"$\rm \dot{M}_{SFR} = 20 M_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.54, 0.59, r"$\rm Z = 4 Z_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.54, 0.54, r"$\rm R_{inject}$ = 300 pc", transform=ax1.transAxes, color="black",  fontsize="large")

ax1.xaxis.set_tick_params(labelsize="large")
ax1.yaxis.set_tick_params(labelsize="large")

ax2.xaxis.set_tick_params(labelsize="large")
ax2.yaxis.set_tick_params(labelsize="large")

ax1.set_xlabel("Radius [kpc]", fontsize=14)
ax1.set_ylabel(r"Density [$\rm cm^{-3}$]", fontsize=14)
plt.tight_layout(w_pad=0.0)

plt.savefig("nd_comp_load_params_v2.pdf", dpi=150, bbox_inches='tight', facecolor='white', transparent=False)
plt.show()

#### Density profiles and phase diagrams.
Plots the temperature profiles from Cell 2 and the temperature-density phase diagrams as a function of mass for 4 snapshots listed in `files`. Was used for the MFP examination.

In [ ]:
import matplotlib.colors as colors

data_init = {}
filename ="./outflow_cooling/output_PIE_fid/snap_000.hdf5"
with h5py.File(filename,'r') as f:
    for key in f['PartType0']:
        data_init[key] = f['PartType0'][key][()]
x_coord_0 = data_init["Coordinates"][:,0] 
y_coord_0 = data_init["Coordinates"][:,1]
z_coord_0 = data_init["Coordinates"][:,2]
abundance_0 = data_init["ElectronAbundance"]
number_density_0 = data_init["Density"]*UnitNumberDensity
internal_energy_0 = data_init["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
temperature_0 = Temp_S(abundance_0, internal_energy_0)
rad_x0, rad_y0, rad_z0 = x_coord_0 - 0.5*boxsize, y_coord_0 - 0.5*boxsize, z_coord_0 - 0.5*boxsize
nd_h = np.percentile(number_density_0[rad_z0 >= 2], 99.85)
t_l = np.percentile(temperature_0[rad_z0 >= 2], 0.15)


halfbox_inner = center_boxsize/2 
lower_bound, upper_bound = midpoint - dx*5, midpoint + dx*5
######### SIMULATION DATA #########
data = {}
files = ["./outflow_cooling/output_PIE_fid/snap_100.hdf5", "./outflow_cooling/output_high_alpha/snap_100.hdf5", "./outflow_cooling/output_high_beta/snap_100.hdf5",
         "./outflow_cooling/output_high_alpha_high_beta/snap_100.hdf5"]

time_labels = [r"t = 50 Myr", "t = 50 Myr", r"t = 50 Myr",r"t = 50 Myr"]
text_label = [r"M82 Disk, $Z = 4 Z_\odot$, $\dot{M}_{sfr} = 20 M_\odot \, \, yr^{-1}$"]

fig = plt.figure(figsize=(12,10.5))
fig.set_rasterized(True)    
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

axs = [ax1, ax2, ax3, ax4]

color_map = plt.get_cmap('turbo')
coloring = color_map(np.linspace(0, 1, len(files)) )

for i, file in enumerate(files): # select the snapshot range to go through
    with h5py.File(file,'r') as f:
        for key in f['PartType0']:
            data[key] = f['PartType0'][key][()]
        header = dict(f['Header'].attrs)
        parameters = dict(f['Parameters'].attrs)
    R = parameters["injection_radius"]
    E_load = parameters["E_load"]
    M_laod = parameters["M_load"]

    coord = data["Coordinates"]
    x_coord = data["Coordinates"][:,0] 
    y_coord = data["Coordinates"][:,1]
    z_coord = data["Coordinates"][:,2]
    density = data["Density"]
    internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
    masses = data["Masses"] 
    vel_x = data["Velocities"][:,0]
    vel_y = data["Velocities"][:,1] 
    vel_z = data["Velocities"][:,2] 
    volume = masses/density
    number_density = density*UnitNumberDensity
    abundance = data["ElectronAbundance"]
    temperature = Temp_S(abundance, internal_energy)
    t = header["Time"]
    times = t*1000
    ''' Get the radial distance of the box'''
    rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
    radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)

    radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) # Units: km/s
    bg_cells = (number_density <= nd_h) & (temperature >= t_l) & (np.abs(radial_velocity) <= 40) # Gets rids of most BG cells.

    edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) # & (radius <= center_boxsize/2*np.sqrt(3))
    theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
   
    angular_region = (np.abs(theta) <= angle_l) & ((np.abs(rad_z) >= R)) & (radius <= 40) # Excludes anything with absolute angles greater than 60 
    angular_bg_mask = (angular_region) & (~bg_cells)   


    volume = masses/density

    log10_nd = np.log10(number_density)
    log10_temperature = np.log10(temperature)
    
    rho_bins = np.linspace(-5, 1, 300)
    T_bins = np.linspace(3, 8, 300)

    td, nd_edge, _ = stats.binned_statistic(log10_nd[angular_bg_mask], log10_temperature[angular_bg_mask], bins=rho_bins, statistic="median")

    stat_phase, xc, zc, bin_n = stats.binned_statistic_2d(log10_nd[angular_bg_mask],
                                                          log10_temperature[angular_bg_mask], 
                                                          masses[angular_bg_mask],
                                                            bins=[rho_bins, T_bins], 
                                                            statistic="sum")

    nd_face = (nd_edge[:-1] + nd_edge[1:])/2
    phase_plot_hist_rho = axs[i].pcolormesh(xc, zc, stat_phase.T, cmap="rocket", shading='auto', norm=colors.LogNorm(vmin=0.01, vmax=1e7))
    labeling.append(r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]))
    nd_profile = axs[i].plot(nd_face, td, color="black", linestyle="dashed", linewidth=2 )

    log10_nd = np.log10(number_density)
    log10_temperature = np.log10(temperature)
    temperate_mean, rho_edge, _ = stats.binned_statistic(log10_nd[angular_region],log10_temperature[angular_region], bins = rho_bins, statistic="median")

    rho_face = (rho_edge[:-1] + rho_edge[1:])/2

    labels_density = [x for x in range(-5, 3, 2)]
    labels_temperature = [x+0.5 for x in range(3,8)]
    labels_P = [x for x in range(1,8, 2)]
    labels_erho = [x for x in range(-27, -16, 2)]

    axs[i].tick_params(axis='both', which='major', labelsize="medium")

    axs[i].set_xticks(labels_density)
    axs[i].set_yticks(labels_temperature)

    axs[i].set(xlim=(-5,1), ylim=(3.0, 8.0))

    axs[i].set_yticks(labels_temperature)
    axs[i].text(-4.9, 7.6, time_labels[i], fontsize=14)
    axs[i].text(-4.9, 7.2, labeling[i], fontsize=14)

plt.subplots_adjust(hspace=0.12, wspace=0.12)

ax1.set_ylabel(r"Temperature [$\rm log_{10}{K}$]", fontsize=14)
ax3.set_ylabel(r"Temperature [$\rm log_{10}{K}$]", fontsize=14)
ax3.set_xlabel(r"Density [$\rm log_{10}(cm^{-3})$]", fontsize=14)
ax4.set_xlabel(r"Density [$\rm log_{10}(cm^{-3})$]", fontsize=14)

fig.subplots_adjust(right=0.84)
cbar_ax = fig.add_axes([0.85, 0.125, 0.018, 0.755])

cbar_masses = fig.colorbar(phase_plot_hist_rho, cax=cbar_ax)
labels_cbar = [1e-3*(10**(x)) for x in range(1,11)]
cbar_masses.set_ticks(labels_cbar)
cbar_masses.set_ticklabels([round(np.log10(label)) for label in (labels_cbar)])
cbar_masses.set_label(r"Masses  [$\rm log_{10}(M_\odot$]", fontsize=14)

plt.rcParams['legend.title_fontsize'] = 'large'

plt.savefig("pd_comp_load_params.pdf", dpi=150, bbox_inches='tight', facecolor='white', transparent=False)


### Velocity
#### Velocity profiles and edge-on slices - Version 1

In [ ]:
from utilities import plot_edge
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

lower_bound, upper_bound = midpoint - dx*4, midpoint + dx*4
######### SIMULATION DATA #########
data = {}
files = ["./outflow_cooling/output_PIE_fid/snap_100.hdf5", "./outflow_cooling/output_high_alpha/snap_100.hdf5", "./outflow_cooling/output_high_beta/snap_100.hdf5",
         "./outflow_cooling/output_high_alpha_high_beta/snap_100.hdf5"]
labeling = []
fig = plt.figure(figsize=(15, 9))
fig.set_rasterized(True)    
gs = GridSpec(2, 3, width_ratios=[1.5, 1, 1], height_ratios=[1, 1], wspace=0.01, hspace=0.01)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 1])
ax5 = fig.add_subplot(gs[1, 2])

axs = [ax2, ax3, ax4, ax5]

color_map = plt.get_cmap('turbo')
coloring = color_map(np.linspace(0, 1, len(files)) )

deviation = 20
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2 + deviation  # boundary of histogram - lower bound


for i, file in enumerate(files): # select the snapshot range to go through
    with h5py.File(file,'r') as f:
        for key in f['PartType0']:
            data[key] = f['PartType0'][key][()]
        header = dict(f['Header'].attrs)
        parameters = dict(f['Parameters'].attrs)
    R = parameters["injection_radius"]
    E_load = parameters["E_load"]
    M_laod = parameters["M_load"]

    coord = data["Coordinates"]
    x_coord = data["Coordinates"][:,0] 
    y_coord = data["Coordinates"][:,1]
    z_coord = data["Coordinates"][:,2]
    density = data["Density"]
    internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
    masses = data["Masses"] 
    vel_x = data["Velocities"][:,0]
    vel_y = data["Velocities"][:,1] 
    vel_z = data["Velocities"][:,2] 
    volume = masses/density
    number_density = density*UnitNumberDensity
    abundance = data["ElectronAbundance"]
    temperature = Temp_S(abundance, internal_energy)
    t = header["Time"]
    times = t*1000
    ''' Get the radial distance of the box'''
    rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
    radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)

    edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) # & (radius <= center_boxsize/2*np.sqrt(3))
    theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
    angular_region = (np.abs(theta) <= angle_l) & (radius <= 20) & ((np.abs(rad_z) >= R))# Excludes anything with absolute angles greater than 60 
    radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) # Units: km/s
    volume = masses/density

    plot_edge(axs[i], coord[edge_mask], radial_velocity[edge_mask], n_bins*2, midpoint, boxsize, 0, 1500, histb_l, histb_h, log=False)
    v_mesh = axs[i].collections[0]
    v_mesh.set_cmap("mako")
    if i % 2 == 0: 
        axs[i].set_ylabel("Z [kpc]", fontsize="medium")
        axs[i].set_yticks([30 + 5*x for x in np.arange(0,8)])
    else: 
        axs[i].set_ylabel("")
        plt.setp(axs[i].get_yticklabels(), visible=False)
    
    if i == 2 or i == 3: 
        axs[i].set_xlabel("X [kpc]", fontsize="medium")
        axs[i].set_xticks([30 + 5*x for x in np.arange(0,8)])
    else: 
        axs[i].set_xlabel("")
        plt.setp(axs[i].get_xticklabels(), visible=False)

    background_rect = patches.Rectangle((0, 0.80), width=1, height=0.2, color='black', alpha=0.25, transform=axs[i].transAxes, fill=True)
    axs[i].add_patch(background_rect)

    axs[i].text(0.03, 0.91, "t = %0.1f Myr" % times, transform=axs[i].transAxes, color="white", fontsize="large")
    axs[i].text(0.03, 0.84, r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]), transform=axs[i].transAxes, color="white", fontsize="large")
    labeling.append(r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]))

axs[0].set_yticks([30 + 5*x for x in np.arange(0,9)])
axs[-1].set_xticks([30 + 5*x for x in np.arange(0,9)])


fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.905, 0.125, 0.012, 0.755])
vcbar = plt.colorbar(v_mesh, cax = cbar_ax, label=r"Radial Velocity [$\rm km/s$]")

for i in np.arange(len(files)):
    ax1.plot(r_face, med_vel[i], label=labeling[i], linestyle="solid", color=coloring[i])
    ax1.fill_between(r_face,  rad_vel_16[i], rad_vel_84[i], color=coloring[i], alpha=0.1)
    ax1.plot(rcwh_face, med_vel_c[i], linestyle="dashed", color=coloring[i])
    ax1.set(xlim=(0, 20), ylim=(0, 2500))
    ax1.set_ylabel(ylabel=(r"Radial Velocity [$\rm km/s$]"), fontsize="large")
    if CC85_PLOTS: ax1.plot(r_an, cc85_vel[i],linestyle="dotted", color=coloring[i])

pos = ax1.get_position()  # get current position [left, bottom, width, height]
ax1.set_position([
    pos.x0,        # keep left position
    pos.ymax/3,        
    pos.width*0.86, 
    pos.height*0.55 
])

l1 = ax1.legend(loc="upper right", ncol=2, fontsize="large", title="Median Radial Velocity(48 - 50 Myr) ",  title_fontsize="large")

l2 = ax1.legend(tcat_labels, tcat_labels, loc ="lower left", fontsize="large")

ax1.add_artist(l1)
ax1.add_artist(l2)

# ax1.set_xticks([0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16, 18])

ax1.text(0.54, 0.69, r"Simulation Parameters", transform=ax1.transAxes, color="black",fontsize="large")
ax1.text(0.54, 0.64, r"$\rm \dot{M}_{SFR} = 20 M_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.54, 0.59, r"$\rm Z = 4 Z_\odot$", transform=ax1.transAxes, color="black",  fontsize="large")
ax1.text(0.54, 0.54, r"$\rm R_{inject}$ = 300 pc", transform=ax1.transAxes, color="black",  fontsize="large")

ax1.xaxis.set_tick_params(labelsize="medium")
ax1.yaxis.set_tick_params(labelsize="medium")

ax1.set_xlabel("Radius [kpc]", fontsize="large")
ax1.set_ylabel(r"Radial Velocity [$\rm km/s$]", fontsize="large")

plt.savefig("radial_velocity_comp_load_params.pdf", dpi=150, bbox_inches='tight', facecolor='white', transparent=False)


#### Velocity profiles and edge-on slices - Version 2

In [ ]:
from utilities import plot_edge
import matplotlib.patches as patches

lower_bound, upper_bound = midpoint - dx*4, midpoint + dx*4
######### SIMULATION DATA #########
data = {}
snapshots_voronoi = "./outflow_cooling/output_PIE_fid/snap_100.hdf5"
fig = plt.figure(figsize=(12, 5))
fig.set_rasterized(True)    
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

deviation = 20
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2 + deviation  # boundary of histogram - lower bound

with h5py.File(snapshots_voronoi,'r') as f:
    for key in f['PartType0']:
        data[key] = f['PartType0'][key][()]
    header = dict(f['Header'].attrs)
    parameters = dict(f['Parameters'].attrs)
    

R = parameters["injection_radius"]
E_load = parameters["E_load"]
M_load = parameters["M_load"]

coord = data["Coordinates"]
x_coord = data["Coordinates"][:,0] 
y_coord = data["Coordinates"][:,1]
z_coord = data["Coordinates"][:,2]
density = data["Density"]
internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
masses = data["Masses"] 
vel_x = data["Velocities"][:,0]
vel_y = data["Velocities"][:,1] 
vel_z = data["Velocities"][:,2] 
volume = masses/density
number_density = density*UnitNumberDensity
abundance = data["ElectronAbundance"]
temperature = Temp_S(abundance, internal_energy)
t = header["Time"]
times = t*1000
''' Get the radial distance of the box'''
rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)

edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) # & (radius <= center_boxsize/2*np.sqrt(3))
theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
angular_region = (np.abs(theta) <= angle_l) & (radius <= 20) & ((np.abs(rad_z) >= R))# Excludes anything with absolute angles greater than 60 

volume = masses/density
#### Velocities - for the center disk plane face####
### PLOTS ###

radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) # Units: km/s
plot_edge(ax2, coord[edge_mask], radial_velocity[edge_mask], n_bins*2, midpoint, boxsize, 0, 1200, histb_l, histb_h, log=False)
rv_mesh = ax2.collections[0]
ax2.set_yticks([30 + 10*x for x in np.arange(0,5)])
ax2.set_xticks([30 + 10*x for x in np.arange(0,5)])

rv_mesh.set_cmap("crest_r")
ax2.set_ylabel("Z [kpc]", fontsize=14)
ax2.set_xlabel("X [kpc]", fontsize=14)

cbar = plt.colorbar(rv_mesh, ax = ax2)
cbar.set_label(r'Radial Velocity [$\rm km/s$]', fontsize=14) 
labels = [0, 200, 400, 600, 800, 1000, 1200]
cbar.set_ticks(labels)

background_rect = patches.Rectangle((0, 0.85), width=1, height=0.15, color='black', alpha=0.25, transform=ax2.transAxes, fill=True)
ax2.add_patch(background_rect)

ax2.text(0.03, 0.95, "t = %0.1f Myr" % times, transform=ax2.transAxes, color="white", fontsize=14) 
ax2.text(0.03, 0.89, r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]), transform=ax2.transAxes, color="white", fontsize=14)
labeling.append(r"$\beta = %.1f, \alpha = %.1f$" % (parameters["M_load"], parameters["E_load"]))

for i in np.arange(len(outputs)):
    if "high_alpha" not in outputs[i]:
        ax1.plot(r_face, med_vel[i], label=labeling[i], linestyle="solid", color=coloring[i])
        ax1.fill_between(r_face, rad_vel_16[i], rad_vel_84[i], color=coloring[i], alpha=0.3)
        ax1.plot(rcwh_face, med_vel_c[i], linestyle="dashed", color=coloring[i])
        ax1.set(xlim=(0.0, 20), ylim=(0, 1600))
        if CC85_PLOTS: ax1.plot(r_an, cc85_vel[i],linestyle="dotted", color=coloring[i])

ax1.set_xlabel("Radius [kpc]", fontsize=14) 
ax1.set_ylabel("Radial Velocity [km/s]", fontsize=14) 

l1 = ax1.legend(loc="upper right", ncol=1, fontsize="large", title="Med. Velocity (48-50 Myr) ",  title_fontsize=14)
l2 = ax1.legend(tcat_labels, tcat_names, loc ="lower right", fontsize=14) 
ax1.add_artist(l1)
ax1.add_artist(l2)

ax1.xaxis.set_tick_params(labelsize="large")
ax1.yaxis.set_tick_params(labelsize="large")
ax2.xaxis.set_tick_params(labelsize="large")
ax2.yaxis.set_tick_params(labelsize="large")
cbar.ax.tick_params(labelsize="large")
plt.tight_layout(w_pad=0.0)

plt.savefig("vel_load_params_updated.pdf", dpi=150, bbox_inches='tight', facecolor='white', transparent=False)
plt.show()

#### Cooling Radii
Calculates the cooling radius for each loading configuration. Note that the $\alpha=1.8, \beta=0.6$ was not included as we do not expect to see a cooling radius for that configuration within our given limits.

In [ ]:
files = ["./outflow_cooling/output_PIE_fid/snap_100.hdf5", "./outflow_cooling/output_high_beta/snap_100.hdf5", "./outflow_cooling/output_high_alpha_high_beta/snap_100.hdf5"]
cr_coloring = ["black", "royalblue", "crimson"]
cr_labeling = [r"$\alpha = 0.9, \beta = 0.6$", r"$\alpha = 0.9, \beta = 1.0$", r"$\alpha = 1.8, \beta = 1.0$" ]
mmw_analytic = 0.6
### ANALYTIC VALUES
rc = np.linspace(0.001, 10, 1500)
r_10 = rc/10
rb = np.linspace(0, 4, 200)
Omega_4pi = (4*np.pi)/(4*np.pi)
t_angle_l = 30

fig = plt.figure(figsize=(6,5))
fig.set_rasterized(True)
ax1 = fig.add_subplot(111)
# ax2 = fig.add_subplot(122)
data = {}
for i, file in enumerate(files): # select the snapshot range to go through
    with h5py.File(file,'r') as f:
        for key in f['PartType0']:
            data[key] = f['PartType0'][key][()]
        header = dict(f['Header'].attrs)
        parameters = dict(f['Parameters'].attrs)
    R = parameters["injection_radius"]
    E_load = parameters["E_load"]
    M_load = parameters["M_load"]
    R = parameters["injection_radius"] # injection radius in kpc
    InitDiskMetallicity = parameters["InitDiskMetallicity"]
    sfr = parameters["sfr"]
    sfr_10 = sfr/10

    coord = data["Coordinates"]
    x_coord = data["Coordinates"][:,0] 
    y_coord = data["Coordinates"][:,1]
    z_coord = data["Coordinates"][:,2]
    density = data["Density"]
    internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
    masses = data["Masses"] 
    vel_x = data["Velocities"][:,0]
    vel_y = data["Velocities"][:,1] 
    vel_z = data["Velocities"][:,2] 
    xe = data["ElectronAbundance"]
    metallicities = data["PassiveScalars"]
    cooling_function = data["CoolingRate"] # Lambda/nh^2 
    z_cooling_funtion = data["MetallicCoolingRate"]
    cooling_time = data["CoolingTime"]
    t = header["Time"]
    temperature = Temp_S(xe, internal_energy)
    linear_velocity = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
    times = t*1000
    volumes = masses/density
    number_density = density*UnitNumberDensity
    density_cgs = density*UnitDensity_in_cgs
    rate_fact = pow(number_density*HYDROGEN_MASS_FRACTION, 2)/density_cgs # cm^-3 * 0.76 

    volumetric_cooling_rate = cooling_function*pow(number_density*HYDROGEN_MASS_FRACTION, 2)
    cooling_rate_erg = cooling_function*rate_fact*masses*UnitMass_in_g # erg cm^3 s^-1 * 

    ''' Get the radial distance of the box'''
    rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
    radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)
    radial_coord = np.sqrt(rad_x**2 + rad_y**2) 
    advection_time_sim = radius*UnitLength_in_cm/(linear_velocity*UnitVelocity_in_cm_per_s)

    ct_ratio = cooling_time*UnitTime_in_s/advection_time_sim

    theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
    angular_region = (np.abs(theta) <= t_angle_l) & (radius <= 15)  # & (radius <= 8)# Excludes anything with absolute angles greater than 60 

    #### Velocities - for the center disk plane face####
    radial_velocity = (vel_x*rad_x + vel_y*rad_y)/(radial_coord + eps) 
    radial_velocity_spherical = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps)
    tvx, tvy = vel_x - radial_velocity*rad_x/(radial_coord+eps), vel_y - radial_velocity*rad_y/(radial_coord+eps)
    tan_velocity = np.sqrt(tvx**2 + tvy**2)

    cooling_ratio, r_edge,  _ = stats.binned_statistic(radius[angular_region], ct_ratio[angular_region], statistic='median', bins=rb)
    ax1.plot(r_edge[:-1], cooling_ratio, label=cr_labeling[i], color=cr_coloring[i])
    advec_tv, advec_r,  _ = stats.binned_statistic(radius[angular_region], advection_time_sim[angular_region], statistic='median', bins=rb)
    cool_time, cool_r,  _ = stats.binned_statistic(radius[angular_region], cooling_time[angular_region]*UnitTime_in_s, statistic='median', bins=rb)

    sfr_10 = sfr/10
    R_03 = R/0.3
    mmw_dimlesss = np.median(mean_molecular_weight(xe))/PROTON_MASS_GRAMS
    cooling_radius = 4 * ((pow(E_load,2.13))/pow(M_load, 2.92) ) * pow(mmw_dimlesss/0.6, 2.13)* pow(R_03, 1.79)* (Omega_4pi/((InitDiskMetallicity/Z_solar)*sfr_10))**0.789
    ax1.axvline(cooling_radius, linestyle="dashed", color=cr_coloring[i])
    advection_time_yr = 1e7 * np.sqrt(M_load/(E_load)) * r_10 # 1e7 years * np.sqrt(M_load/(E_load*mu_t)) * r_10
    cooling_time_yr = 3e6 * (((E_load*0.6)**2.20)/(M_load**3.20)) * (R_03/r_10)**0.27 * R_03**2/sfr_10*(Omega_4pi/(InitDiskMetallicity/Z_solar)) # /sqrt(mmw_analytic) # 3e6 years * * (((E_load*mu_t)**2.20)/(M_load**3.20)) * (R_03/r_10)**0.27 * R_03**2/sfr_10*Omega_4pi

ax1.axhline(1, linestyle="dotted", color="black")
ax1.set(xlim=(0, 4), ylim=(0,4))
ax1.set_ylabel(r'Cooling Time/Advection Time', fontsize=14)
ax1.set_xlabel(r"Radius [kpc]", fontsize=14)
ax1.legend(loc='upper right', fontsize=14)

ax1.tick_params(axis='both', which='major', labelsize="large")
ax1.tick_params(axis='both', which='minor', labelsize="large")


plt.savefig("cooling_radii.pdf", bbox_inches='tight', dpi=150)